# Applying data minimization to a trained regression ML model

In this tutorial we will show how to perform data minimization for regression ML models using the minimization module.

We will show you applying data minimization to a different trained regression models.

## Load data
QI parameter determines which features will be minimized.

In [20]:
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split

dataset = load_diabetes()
X_train, X_test, y_train, y_test = train_test_split(dataset.data, dataset.target, test_size=0.5, random_state=14)

features = ['age', 'sex', 'bmi', 'bp',
                's1', 's2', 's3', 's4', 's5', 's6']
QI = ['age', 'bmi', 's2', 's5', 's6']

## Train DecisionTreeRegressor model

In [21]:
import os
import sys
apt_path = os.path.abspath("/Users/eliandromelo/MSc_CS/ai-privacy-toolkit")
sys.path.append(apt_path)

import warnings
warnings.filterwarnings("ignore")

from apt.minimization import GeneralizeToRepresentative
from sklearn.tree import DecisionTreeRegressor

model1 = DecisionTreeRegressor(random_state=10, min_samples_split=2)
model1.fit(X_train, y_train)
print('Base model accuracy (R2 score): ', model1.score(X_test, y_test))

Base model accuracy (R2 score):  0.15014421352446072


## Run minimization
We will try to run minimization with only a subset of the features.

In [22]:
# note that is_regression param is True

minimizer1 = GeneralizeToRepresentative(model1, target_accuracy=0.7, is_regression=True,
                                    features_to_minimize=QI)

# Fitting the minimizar can be done either on training or test data. Doing it with test data is better as the
# resulting accuracy on test data will be closer to the desired target accuracy (when working with training
# data it could result in a larger gap)
# Don't forget to leave a hold-out set for final validation!
X_generalizer_train1, x_test1, y_generalizer_train1, y_test1 = train_test_split(X_test, y_test,
                                                                test_size = 0.4, random_state = 38)

x_train_predictions1 = model1.predict(X_generalizer_train1)
minimizer1.fit(X_generalizer_train1, x_train_predictions1, features_names=features)
transformed1 = minimizer1.transform(x_test1, features_names=features)
print('Accuracy on minimized data: ', model1.score(transformed1, y_test1))
print('generalizations: ',minimizer1.generalizations)#%% md

Initial accuracy of model on generalized data, relative to original model predictions (base generalization derived from tree, before improvements): 0.108922
Improving accuracy
feature to remove: s5
Removed feature: s5, new relative accuracy: 0.505498
feature to remove: bmi
Removed feature: bmi, new relative accuracy: 0.716972
Accuracy on minimized data:  0.1116122925781402
generalizations:  {'ranges': {'age': [-0.07090024650096893, -0.043656209483742714, -0.041839939542114735, -0.03639113181270659, -0.01459590089507401, -0.012779632292222232, -0.009147093165665865, -0.0036982858437113464, 0.03989217430353165, 0.039892176166176796, 0.05623859912157059, 0.06713621318340302], 's2': [-0.0550188384950161, -0.0285577941685915, -0.024643437936902046, -0.02135537937283516, -0.013683241792023182, -0.006480826530605555, 0.009176596067845821, 0.023111702874302864, 0.02420772146433592, 0.02655633445829153, 0.039082273840904236], 's6': [-0.052854035049676895, -0.03835666086524725, -0.02593033987795

## Train linear regression model

In [3]:
from concrete.ml.sklearn.linear_model import LinearRegression
from apt.minimization import GeneralizeToRepresentative
from sklearn.metrics import r2_score

model2 = LinearRegression()
model2.fit(X_train, y_train)
model2.compile(X_train[:100])
y_pred = model2.predict(X_test, fhe="execute")
print('Base model accuracy (R2 score): ', r2_score(X_test, y_pred))

AttributeError: dlsym(0xec760690, path): symbol not found

## Run minimization
We will try to run minimization with only a subset of the features.

In [24]:
# note that is_regression param is True

minimizer2 = GeneralizeToRepresentative(model2, target_accuracy=0.7, is_regression=True,
                                    features_to_minimize=QI)

# Fitting the minimizar can be done either on training or test data. Doing it with test data is better as the
# resulting accuracy on test data will be closer to the desired target accuracy (when working with training
# data it could result in a larger gap)
# Don't forget to leave a hold-out set for final validation!
X_generalizer_train2, x_test2, y_generalizer_train2, y_test2 = train_test_split(X_test, y_test,
                                                                test_size = 0.4, random_state = 38)

x_train_predictions2 = model2.predict(X_generalizer_train2)
minimizer2.fit(X_generalizer_train2, x_train_predictions2, features_names=features)
transformed2 = minimizer2.transform(x_test2, features_names=features)
print('Accuracy on minimized data: ', model2.score(transformed2, y_test2))
print('generalizations: ',minimizer2.generalizations)

Initial accuracy of model on generalized data, relative to original model predictions (base generalization derived from tree, before improvements): -0.030344
Improving accuracy
feature to remove: s5
Removed feature: s5, new relative accuracy: 0.146959
feature to remove: s6
Removed feature: s6, new relative accuracy: 0.131751
feature to remove: age
Removed feature: age, new relative accuracy: 0.120796
feature to remove: s2
Removed feature: s2, new relative accuracy: 0.962978
Accuracy on minimized data:  0.46500149833646587
generalizations:  {'ranges': {'bmi': [-0.0660245232284069, -0.06171327643096447, -0.048779530450701714, -0.036923596635460854, -0.022912041284143925, -0.01644516922533512, -0.015906263142824173, -0.009978296235203743, 0.007266696775332093, 0.022356065921485424, 0.028822937980294228, 0.04499012045562267, 0.04876246117055416, 0.053073709830641747, 0.09510837681591511, 0.10103634744882584]}, 'categories': {}, 'untouched': ['sex', 'age', 'bp', 's2', 's1', 's6', 's4', 's5'